<h1>Testing Naive Bayes on students Test Scores</h1>


Data from: https://www.kaggle.com/spscientist/students-performance-in-exams

This was used with the help of : https://www.becomingadatascientist.com/learningclub/

We want to look at the dataset of student performance on exams and try to see which variables have the most impact on exam scores. We also want to see if we could run a Naive Bayes algorithim on the data and see how accurate it could be.

First we will import all of the necessary pacakges and data

In [ ]:
#import pandas and numpy libraries
import pandas as pd
import numpy as np
import sys #sys needed only for python version
#import gaussian naive bayes from scikit-learn
import sklearn as sk
#seaborn for pretty plots
import seaborn as sns
df=pd.read_csv('../input/students-performance-in-exams/StudentsPerformance.csv')

# set pandas to output all of the columns in output
pd.options.display.max_columns = 25
#show the first 5 rows
print(df.head(n=5))

I want to find out all the possible values that each column can contain. We can use histograms to figure out the unique values and the distribution


<h2> Pre-liminary Analysis of All Variable</h2>

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib inline


for column in df.columns:
#     print('\n'+'This is a summary df for ' + column)
#     x = (df[column].value_counts(sort=True,normalize=False))
    plt.hist(df[column])
    plt.title(column)
    plt.xticks(rotation=30)
    plt.show()

Looking at the three histograms for math, writing and reading it seems like these are fairly normally distributed, but I want to use a box plot to get some summary statistics and then establish a threshold for passing
<h2> Test Scores Distribution</h2>

In [ ]:
testScores=df.columns[5:8]
for count,score in enumerate(testScores):
    plt.subplot(1,3,count+1)
    plt.boxplot(df[score])
    plt.title(score)
    plt.subplots_adjust(right =1.9)
    print("\n The 25, 50 and 75 percentiles are as follows: ")
    print(np.percentile(df[score],[25,50,75]))

The data seems to have a fairly similar spread.
We will say that anyone above the 25th percentile passed and everyone below failed. Now we have to classify these into new columns

In [ ]:
for score in testScores:
    name =score + " pass"
    df[name]=(df[score]>(np.percentile(df[score],[25]))[0])*1
#Now I want to make sure that all students pass all three tests and set that to pass for if they pass all three and fail if they don't
df['passed all']=df["math score pass"]+df["reading score pass"]+df["writing score pass"]
# df.loc[df['passed all']==3]="pass"
# df.loc[df['passed all']!=3]="fail"
df.loc[(df['passed all'] != 3),'passed all']='failed'
df.loc[(df['passed all'] == 3),'passed all']='passed'


Now we will plot if the student based compared to other factors
<h2> Pass Fail Based Factor Plots </h2>

In [ ]:
#seaborn factorplot to show pass/noPass all tests by different factors
df_forplot =df.loc[:,('passed all', 'gender','race/ethnicity','parental level of education','lunch','test preparation course')]
g = sns.catplot("passed all", col="gender", data=df_forplot,kind="count", height=2.5, aspect=.8, col_wrap=2)
r = sns.catplot("passed all", col="race/ethnicity", data=df_forplot,kind="count", height=2.5, aspect=.8, col_wrap=5)
p = sns.catplot("passed all", col="parental level of education", data=df_forplot,kind="count", height=2.5, aspect=1.4, col_wrap=6)
l = sns.catplot("passed all", col="lunch", data=df_forplot,kind="count", height=2.5, aspect=.8, col_wrap=2)
t = sns.catplot("passed all", col="test preparation course", data=df_forplot,kind="count", height=2.5, aspect=1.2, col_wrap=2)

The most striking data that comes out of this is that the race ethnicity group A seems to have much lower passing rates compared to everyone else. This appears to be a minority group given by the little number of total people. It also seems that more people on free/reduced lunch failed than normal. Perhaps signifying a correlation between income and passing the tests. It also seems like people who complete test prep courses are more likely to pass. 

<h1> Naive Bayes Classification</h1>

In [ ]:
#Need to find a better way to pull the columns
X = pd.DataFrame(df, columns=(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course']), index=df.index)
    #Can also use the drop function
    #df.drop(['passed all','reading score pass', 'math score pass','writing score pass',... other columns],axis=1)
#this is the outcome column
Y = df['passed all']

#encode the text category labels as numeric
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
#Y values now boolean values; poison = 1
y = le.transform(Y)
#print(y_train)

#have to initialize or get error below
x = pd.DataFrame(X,columns=[X.columns[0]])
#encode each feature column and add it to x_train
#makes every feature into a number
for colname in X.columns:
    le.fit(X[colname])
    #print(colname, le.classes_)
    x[colname] = le.transform(X[colname])
#split the dataset into training and test sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

#initialize and fit the naive bayes classifier
from sklearn.naive_bayes import GaussianNB
skgnb = GaussianNB()
skgnb.fit(x_train,y_train)
train_predict = skgnb.predict(x_train)
#print(train_predict)

#see how accurate the training data was fit
from sklearn import metrics
print("Training accuracy:",metrics.accuracy_score(y_train, train_predict))

#use the trained model to predict the test values
test_predict = skgnb.predict(x_test)
print("Testing accuracy:",metrics.accuracy_score(y_test, test_predict))

Happy to take any feedback/criticism.